# Self-Organizing Maps - SOMs

### Organizing animals

In [1]:
import numpy as np

In [2]:
data = np.genfromtxt('data_lab2/animals.dat',
                     dtype=None,
                     delimiter=',')
data = data.reshape(32,84)
data.shape

(32, 84)

In [33]:
data[0]

array([1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1,
       0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0])

### Algorithm: for each sample
1. Calculate the similarity between the input pattern and the weights arriving
at each output node.
2. Find the most similar node; often referred to as the winner.
3. Select a set of output nodes which are located close to the winner in the
output grid. This is called the neighbourhood.
4. Update the weights of all nodes in the neighbourhood such that their
weights are moved closer to the input pattern.

In [34]:
def descending_logarithmic(x, n):
    A = 49
    k = -np.log(0.03) / 16  # Calcola k in modo che y sia 1 quando x è 16
    B = 0  # Per ottenere y = 1 quando x è 16
    return A * np.exp(-k * x) + B

In [35]:
def custom_sigmoid(x):
    k = 4  # Puoi regolare k per controllare la pendenza
    return 50 - (50 / (1 + np.exp(-k * (x - 15))))

In [36]:
import math

class SOM:
    def __init__(self, output_shape, data_dimensionality, step_size, neighborhood_size):
        self.output_shape = output_shape
        self.weight_matrix = np.random.random((output_shape, data_dimensionality))
        # self.weight_matrix = np.ones((output_shape, data_dimensionality))
        self.step_size = step_size
        self.neighborhood_size = neighborhood_size

    def find_winner(self, sample):
        distances = []
        for row in range(self.weight_matrix.shape[0]):
            d = np.linalg.norm(sample - self.weight_matrix[row,:])
            distances.append(d)
            # print("{}: {}".format(row, d))
        
        winner = np.argmin(np.array(distances))
        # print("winner is: {}".format(winner))
        return winner

    def get_neighborhood(self, winner):
        neighborhood = range(np.clip(winner - self.neighborhood_size, 0, None), np.clip(winner + self.neighborhood_size, None, self.output_shape - 1))
        return neighborhood
    
    def update_weights(self, neighborhood, sample):
        for row in range(self.weight_matrix.shape[0]):
            if row in neighborhood:
                # print("{} is in neighbors".format(row))
                # print("old row{}: {}".format(row, self.weight_matrix[row]))
                new_row = self.weight_matrix[row] + (sample - self.weight_matrix[row]) * self.step_size
                self.weight_matrix[row] = new_row
                # print("new_row {}: {}".format(row, new_row))

    def fit(self, X, n_epochs):
        for e in range(n_epochs):
            # y = -3*e + 50
            y = descending_logarithmic(e)
            # y = custom_sigmoid(e)

            if (y>0):
                self.neighborhood_size = math.floor(y)
            else:
                self.neighborhood_size = 0

            print("{}: {}".format(e, self.neighborhood_size))

            for row in range(X.shape[0]):
                winner = self.find_winner(X[row])
                neighborhood = self.get_neighborhood(winner)

                self.update_weights(neighborhood, X[row])

In [37]:
som = SOM(100, 84, 0.2, 50)

In [38]:
som.fit(data, 20)

0: 49
1: 39
2: 31
3: 25
4: 20
5: 16
6: 13
7: 10
8: 8
9: 6
10: 5
11: 4
12: 3
13: 2
14: 2
15: 1
16: 1
17: 1
18: 0
19: 0


In [39]:
som.weight_matrix

array([[2.73871761e-03, 1.61755571e-13, 1.72402375e-08, ...,
        6.29742983e-14, 1.26781860e-12, 5.87237291e-01],
       [4.22995265e-01, 9.50646454e-14, 1.37297527e-08, ...,
        1.33917766e-14, 8.03388599e-13, 1.79467883e-01],
       [7.12217495e-01, 1.21044704e-13, 2.68131917e-08, ...,
        4.13757217e-14, 1.03852567e-12, 1.12220159e-01],
       ...,
       [3.90380339e-08, 9.99999783e-01, 3.19933428e-08, ...,
        3.13322407e-04, 9.99686572e-01, 5.56814134e-03],
       [6.08984506e-08, 9.99999823e-01, 4.01276463e-08, ...,
        3.13292110e-04, 9.99686629e-01, 5.56812261e-03],
       [6.62486690e-01, 8.13865839e-01, 1.77939593e-02, ...,
        3.29701119e-02, 8.53232302e-01, 6.43724927e-01]])

In [40]:
som.find_winner(data[0])

2

In [41]:
winnrz = []

for row in range(data.shape[0]):
    winnrz.append( (row, som.find_winner(data[row])) )

In [42]:
len(winnrz), winnrz

(32,
 [(0, 2),
  (1, 32),
  (2, 15),
  (3, 38),
  (4, 95),
  (5, 89),
  (6, 0),
  (7, 28),
  (8, 47),
  (9, 22),
  (10, 98),
  (11, 65),
  (12, 7),
  (13, 54),
  (14, 0),
  (15, 92),
  (16, 5),
  (17, 81),
  (18, 35),
  (19, 9),
  (20, 25),
  (21, 85),
  (22, 58),
  (23, 70),
  (24, 61),
  (25, 0),
  (26, 12),
  (27, 17),
  (28, 50),
  (29, 19),
  (30, 77),
  (31, 43)])

In [43]:
sorted_winnrz = sorted(winnrz, key=lambda x: x[1])
sorted_winnrz

[(6, 0),
 (14, 0),
 (25, 0),
 (0, 2),
 (16, 5),
 (12, 7),
 (19, 9),
 (26, 12),
 (2, 15),
 (27, 17),
 (29, 19),
 (9, 22),
 (20, 25),
 (7, 28),
 (1, 32),
 (18, 35),
 (3, 38),
 (31, 43),
 (8, 47),
 (28, 50),
 (13, 54),
 (22, 58),
 (24, 61),
 (11, 65),
 (23, 70),
 (30, 77),
 (17, 81),
 (21, 85),
 (5, 89),
 (15, 92),
 (4, 95),
 (10, 98)]

In [44]:
names = np.genfromtxt('data_lab2/animalnames.txt',
                     dtype=str)
names

array(["'antelop'", "'ape'", "'bat'", "'bear'", "'beetle'", "'butterfly'",
       "'camel'", "'cat'", "'crocodile'", "'dog'", "'dragonfly'",
       "'duck'", "'elephant'", "'frog'", "'giraffe'", "'grasshopper'",
       "'horse'", "'housefly'", "'hyena'", "'kangaroo'", "'lion'",
       "'moskito'", "'ostrich'", "'pelican'", "'penguin'", "'pig'",
       "'rabbit'", "'rat'", "'seaturtle'", "'skunk'", "'spider'",
       "'walrus'"], dtype='<U13')

In [45]:
for w in sorted_winnrz:
    print(names[w[0]], w[1])

'camel' 0
'giraffe' 0
'pig' 0
'antelop' 2
'horse' 5
'elephant' 7
'kangaroo' 9
'rabbit' 12
'bat' 15
'rat' 17
'skunk' 19
'dog' 22
'lion' 25
'cat' 28
'ape' 32
'hyena' 35
'bear' 38
'walrus' 43
'crocodile' 47
'seaturtle' 50
'frog' 54
'ostrich' 58
'penguin' 61
'duck' 65
'pelican' 70
'spider' 77
'housefly' 81
'moskito' 85
'butterfly' 89
'grasshopper' 92
'beetle' 95
'dragonfly' 98
